<a href="https://colab.research.google.com/github/sachinthadilshann/Temp_Forcasting/blob/main/Temp_ARIAM__70_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/SLT/Se/temp.csv", sep=';')

In [ ]:
df.head()

In [ ]:
df[['id', 'device_id', 'sensor_value', 'timestamp']] = df['id;"device_id";"sensor_value";"timestamp"'].str.split(';', expand=True)

In [ ]:
df.head()

In [ ]:
df['device_id'] = df['device_id'].str.strip('"')
df['sensor_value'] = df['sensor_value'].str.strip('"')
df['timestamp'] = df['timestamp'].str.strip('"')

In [ ]:
df.drop('id;"device_id";"sensor_value";"timestamp"', axis=1, inplace=True)


In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['sensor_value'].astype(float)

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.sort_values(by='timestamp', inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

In [ ]:
df['sensor_value'] = df['sensor_value'].astype(float)

In [ ]:
df['timestamp'].value_counts()

In [ ]:
df['device_id'] = df['device_id'].replace({"Device0001a": 1, "Device0002a": 0})


In [ ]:
df['device_id'].value_counts()

In [ ]:
df = df.drop(df[(df['device_id'] == 0)].index)

In [ ]:
df.head(100)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.drop('device_id', axis=1, inplace=True)

In [ ]:
df.head(300)

In [ ]:
df.set_index('timestamp', inplace=True)

In [ ]:
df.shape

In [ ]:
start_timestamp = '2023-07-12 08:40:00'
end_timestamp = '2023-07-12 09:30:27'
df = df.loc[start_timestamp:end_timestamp]


plt.figure(figsize=(20, 8))
plt.plot(df.index, df['sensor_value'])
plt.xlabel('Timestamp')
plt.ylabel('Sensor Value')
plt.title('Sensor Value over Time')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
#df = df.iloc[1000:]
#df = df.iloc[:-1000]

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.head(3000)

In [ ]:
#df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.shape

In [ ]:
df.plot(figsize=(20, 8))

In [ ]:
#df = df.resample('30S').mean()


In [ ]:
#df['sensor_value'] = df['sensor_value'].fillna(0)


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.plot(figsize=(30, 20))

In [ ]:
df.isnull().sum()

In [ ]:


from statsmodels.tsa.stattools import adfuller

def adf_test(dataset):
  dftest = adfuller(dataset, autolag = 'AIC')
  print("1. ADF : ",dftest[0])
  print("2. P-Value : ", dftest[1])
  print("3. Num Of Lags : ", dftest[2])
  print("4. Num Of Observations Used For ADF Regression and Critical Values Calculation :", dftest[3])
  print("5. Critical Values :")
  for key, val in dftest[4].items():
      print("\t",key, ": ", val)



In [ ]:
adf_test(df['sensor_value'])


In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima import auto_arima



In [ ]:
stepwise_fit = auto_arima(df['sensor_value'],trace = True, suppress_warnings=True,stepwise = False,seasonal = False)
stepwise_fit.summary()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

#model = ARIMA(df, order=(2, 1, 5))
#model_fit = model.fit()

# Print the model summary
#print(model_fit.summary())


In [ ]:
#forecast = model_fit.forecast(steps=50)
#print(forecast)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA


# Extract the data values
X = df['sensor_value'].values

# Split the data into training and testing sets (80% training, 20% testing)
size = int(len(X) * 0.7)
train, test = X[:size], X[size:]

# Initialize a list to store the training data
history = [x for x in train]

# Initialize a list to store the predicted values
predictions = []

# Loop through the testing set to make predictions
for t in range(len(test)):
    model = ARIMA(history, order=(3, 1, 1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print(f'Predicted: {yhat:.2f}, Expected: {obs:.2f}')

# Create a DataFrame for the predicted values with timestamps



In [ ]:
pred_df = pd.DataFrame(predictions, index=df.index[size:size+len(test)], columns=['predicted_sensor_value'])

In [ ]:
pred_df

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(test, color='blue', label='Actual')
plt.plot(predictions, color='red', label='Predicted')
plt.xlabel('Timestamp')
plt.ylabel('Sensor Value')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)
